## k-line joins

### First we will try to implement the 4-line join specified in the Probplem statement

In [2]:
def semijoin(relation1, relation2, key1, key2):
    """ Reduce relation1 by performing a semijoin with relation2 on specified keys. """
    filter_set = {t[key2] for t in relation2}
    return [t for t in relation1 if t[key1] in filter_set]

In [3]:
def join(relation1, relation2, key1, key2):
    """ Join two relations on specified keys, handling duplicates appropriately. """
    index = {}
    # Create an index for the second relation on the join key
    for t in relation2:
        if t[key2] in index:
            index[t[key2]].append(t)
        else:
            index[t[key2]] = [t]
    
    # Perform the join
    result = []
    for t1 in relation1:
        if t1[key1] in index:
            for t2 in index[t1[key1]]:
                # Concatenate tuples while removing the repeated join attribute from the second tuple
                result.append(t1 + t2[1:])  # Skip the join attribute of the second tuple
    return result

In [4]:
# Relations Initialization
R1 = [(1, 2), (3, 4)]
R2 = [(2, 5), (4, 6)]
R3 = [(5, 7), (6, 8)]

# Perform Semijoin Reductions
R1_reduced = semijoin(R1, R2, 1, 0)
R2_reduced = semijoin(R2, R3, 1, 0)

# Perform Joins
result1 = join(R1_reduced, R2_reduced, 1, 0)
final_result = join(result1, R3, 2, 0)

# Strip redundant elements from the join result
# This adjustment ensures each tuple has the intended format and no repeated attributes
final_result_cleaned = [(a1, a2, a3, a4) for (a1, a2, a3, a4) in final_result]

print("Final Result:")
for line in final_result_cleaned:
    print(line)

Final Result:
(1, 2, 5, 7)
(3, 4, 6, 8)


### Next lets generalize it for k-line queries.

In [5]:
class K_Line_Joins:
    def semijoin(self, relation1, relation2, key1, key2):
        """ Reduce relation1 by performing a semijoin with relation2 on specified keys. """
        filter_set = {t[key2] for t in relation2}
        return [t for t in relation1 if t[key1] in filter_set]

    def join(self, relation1, relation2, key1, key2):
        """ Join two relations on specified keys, handling duplicates appropriately. """
        index = {}

        # Create an index for the second relation on the join key
        for t in relation2:
            if t[key2] in index:
                index[t[key2]].append(t)
            else:
                index[t[key2]] = [t]
        # print(index)

        # Perform the join
        result = []
        for t1 in relation1:
            if t1[key1] in index:
                for t2 in index[t1[key1]]:
                    # Concatenate tuples while removing the repeated join attribute from the second tuple
                    result.append(t1 + t2[1:])  # Skip the join attribute of the second tuple

        return result

    def k_line_join(self, relations):
        """ Perform a k-line join on a list of relations using the specified keys. """
        if not relations:
            return []

        # Start with the first relation
        result = relations[0]

        # Perform successive joins
        for i in range(1, len(relations)):
            result = self.join(result, relations[i], i, 0)
            # print(result)

        return result
    def process_and_join(self, R):
        R_reduced = []
        for i in range(len(R) - 1):
            reduced = self.semijoin(R[i], R[i+1], 1, 0)
            R_reduced.append(reduced)
        R_reduced.append(R[-1])
        final_result = self.k_line_join(R_reduced)
        return final_result

## Test cases - 
### 1. Simple Line Join Query

In [6]:
R1 = [(1, 2), (3, 4), (5, 6)]
R2 = [(2, 7), (4, 8), (6, 9)]
R3 = [(7, 10), (8, 11), (9, 12)]
R = [
    R1, 
    R2, 
    R3
    # R4, 
    # R5,
    # R6,
    # R7,
    # R8,
    # R9,
    # R10
]
k_line_joiner = K_Line_Joins()
final_result = k_line_joiner.process_and_join(R)
print("Final Result:")
for line in final_result:
    print(line)

Final Result:
(1, 2, 7, 10)
(3, 4, 8, 11)
(5, 6, 9, 12)


### 3. Line Join Query with Empty Relations

In [7]:
R1 = [(1, 2), (3, 4)]
R2 = []
R3 = [(7, 10), (8, 11)]
R = [
    R1, 
    R2, 
    R3
    # R4, 
    # R5,
    # R6,
    # R7,
    # R8,
    # R9,
    # R10
]
k_line_joiner = K_Line_Joins()
final_result = k_line_joiner.process_and_join(R)
print("Final Result:")
for line in final_result:
    print(line)

Final Result:


### 3. Line Join Query with No Matching Tuples:

In [8]:
R1 = [(1, 2), (3, 4)]
R2 = [(5, 6), (7, 8)]
R3 = [(9, 10), (11, 12)]
R = [
    R1, 
    R2, 
    R3
    # R4, 
    # R5,
    # R6,
    # R7,
    # R8,
    # R9,
    # R10
]
k_line_joiner = K_Line_Joins()
final_result = k_line_joiner.process_and_join(R)
print("Final Result:")
for line in final_result:
    print(line)

Final Result:


### 4. Tuples with negative values

In [16]:
R1 = [(0, 1), (1, 2), (2, 3)]
R2 = [(1, 4), (2, 5), (3, 6)]
R3 = [(4, -1), (5, -2), (6, -3)]

R = [
    R1, 
    R2, 
    R3
    # R4, 
    # R5,
    # R6,
    # R7,
    # R8,
    # R9,
    # R10
]
k_line_joiner = K_Line_Joins()
final_result = k_line_joiner.process_and_join(R)
print("Final Result:")
for line in final_result:
    print(line)

Final Result:
(0, 1, 4, -1)
(1, 2, 5, -2)
(2, 3, 6, -3)


### 5. Line Join Query with Duplicate Tuples:

In [9]:
R1 = [(1, 2), (3, 4), (1, 2)]
R2 = [(2, 5), (4, 6), (2, 5)]
R3 = [(5, 7), (6, 8), (5, 7)]
R = [
    R1, 
    R2, 
    R3
    # R4, 
    # R5,
    # R6,
    # R7,
    # R8,
    # R9,
    # R10
]
k_line_joiner = K_Line_Joins()
final_result = k_line_joiner.process_and_join(R)
print("Final Result:")
for line in final_result:
    print(line)

Final Result:
(1, 2, 5, 7)
(1, 2, 5, 7)
(1, 2, 5, 7)
(1, 2, 5, 7)
(3, 4, 6, 8)
(1, 2, 5, 7)
(1, 2, 5, 7)
(1, 2, 5, 7)
(1, 2, 5, 7)


### Time Complexity Analysis:
Our solution consists of two main parts:

1. The `semijoin` function:
   - This function performs a semijoin reduction by filtering the tuples in `relation1` based on the values in `relation2`.
   - The time complexity of this function is O(M + N), where M is the size of `relation1`, and N is the size of `relation2`.

2. The `join` function:
   - This function performs a hash-based join between two relations.
   - The time complexity of building the hash index is O(N), where N is the size of `relation2`.
   - The time complexity of the join operation is O(M), where M is the size of `relation1`.
   - Therefore, the overall time complexity of the `join` function is O(M + N).

3. The `k_line_join` function:
   - This function performs successive joins on the list of relations.
   - The time complexity of this function is O(N * (k - 1) + OUT), where N is the size of each relation, k is the number of relations, and OUT is the size of the output.

4. The `process_and_join` function:
   - This function performs semijoin reductions on the relations before calling `k_line_join`.
   - The time complexity of the semijoin reductions is O(N * (k - 1)), where N is the size of each relation, and k is the number of relations.
   - The time complexity of `k_line_join` is O(N * (k - 1) + OUT), where OUT is the size of the output.
   - Therefore, the overall time complexity of `process_and_join` is O(N * (k - 1) + OUT).

In the given problem statement, the assumption is that each relation has N tuples, and the domain of each attribute is Z (the set of integer numbers). Additionally, the focus is on k-line join queries for k ≤ 10.

Under these assumptions, the time complexity of your solution is O(N * (k - 1) + OUT), which satisfies the requirement of O(N + OUT) time complexity for k-line join queries.

### Correctness Analysis:

1. The `semijoin` function correctly filters the tuples in `relation1` based on the values in `relation2`.
2. The `join` function correctly performs a hash-based join between two relations, handling duplicates appropriately.
3. The `k_line_join` function correctly performs successive joins on the list of relations, using the `join` function.
4. The `process_and_join` function correctly performs semijoin reductions on the relations before calling `k_line_join`.

Our solution handles various test cases, including simple line join queries, queries with empty relations, queries with no matching tuples, tuples with negative values, and queries with duplicate tuples. The output matches the expected results in all these test cases.